# Assignment 3 - Text Analysis
An explanation this assignment could be found in the .pdf explanation document


## Materials to review for this assignment
<h4>From Moodle:</h4> 
<h5><u>Review the notebooks regarding the following python topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Working with strings</b> (tutorial notebook)<br/>
&#x2714; <b>Text Analysis</b> (tutorial notebook)<br/>
&#x2714; <b>Hebrew text analysis tools (tokenizer, wordnet)</b> (moodle example)<br/>
&#x2714; <b>(brief review) All previous notebooks</b><br/>
</div> 
<h5><u>Review the presentations regarding the following topics</u>:</h5>
<div class="alert alert-info">
&#x2714; <b>Text Analysis</b> (lecture presentation)<br/>
&#x2714; <b>(brief review) All other presentations</b><br/>
</div>

## Personal Details:

In [ ]:
# Details Student 1:
Dor Nagaoker 



## Preceding Step - import modules (packages)
This step is necessary in order to use external modules (packages). <br/>

In [20]:
# --------------------------------------
import pandas as pd
import numpy as np
# --------------------------------------


# --------------------------------------
# ------------- visualizations:
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
# --------------------------------------


# ---------------------------------------
import sklearn
from sklearn import preprocessing, metrics, pipeline, model_selection, feature_extraction 
from sklearn import naive_bayes, linear_model, svm, neural_network, neighbors, tree
from sklearn import decomposition, cluster

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV 
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.metrics import mean_squared_error, r2_score, silhouette_score
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder

from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import Perceptron, SGDClassifier
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
# ---------------------------------------


# ----------------- output and visualizations: 
import warnings
from sklearn.exceptions import ConvergenceWarning
warnings.simplefilter("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter("ignore", category=ConvergenceWarning)
# show several prints in one cell. This will allow us to condence every trick in one cell.
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline
pd.pandas.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)
# ---------------------------------------

### Text analysis and String manipulation imports:

In [21]:
# --------------------------------------
# --------- Text analysis and Hebrew text analysis imports:
# vectorizers:
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# regular expressions:
import re
# --------------------------------------

### (optional) Hebrew text analysis - WordNet (for Hebrew)
Note: the WordNet is not a must

#### (optional) Only if you didn't install Wordnet (for Hebrew) use:

In [22]:
# word net installation:
# unmark if you want to use and need to install
!pip install wn
!python -m wn download omw-he:1.4




Cached file found: C:\Users\dor89\.wn_data\downloads\7ecf10e89326bc0ac26ad94b40fe60a7b6ac3302

Checking C:\Users\dor89\AppData\Local\Temp\tmpw0c7rjnb\omw-he\omw-he.xml
Skipping omw-he:1.4 (Hebrew Wordnet); already added



In [23]:
# word net import:

# unmark if you want to use:
import wn

### (optional) Hebrew text analysis - hebrew_tokenizer (Tokenizer for Hebrew)
Note: the hebrew_tokenizer is not a must

#### (optional) Only if you didn't install hebrew_tokenizer use:

In [24]:
# Hebrew tokenizer installation:

# unmark if you want to use and need to install:
!pip install hebrew_tokenizer

In [25]:
# Hebrew tokenizer import:

# unmark if you want to use:
import hebrew_tokenizer as ht

### Reading input files
Reading input files for train annotated corpus (raw text data) corpus and for the test corpus

In [26]:
train_filename = 'annotated_corpus_for_train.csv'
test_filename  = 'corpus_for_test.csv'
df_train = pd.read_csv(train_filename, index_col=None, encoding='utf-8')
df_test  = pd.read_csv(test_filename, index_col=None, encoding='utf-8')

In [27]:
df_train.head(8)
df_train.shape

,story,gender
0,"כשחבר הזמין אותי לחול, לא באמת חשבתי שזה יקרה,...",m
1,לפני שהתגייסתי לצבא עשיתי כל מני מיונים ליחידו...,m
2,מאז שהתחילו הלימודים חלומו של כל סטודנט זה הפנ...,f
3,"כשהייתי ילד, מטוסים היה הדבר שהכי ריתק אותי. ב...",m
4,‏הייתי מדריכה בכפר נוער ומתאם הכפר היינו צריכי...,f
5,לפני כ3 חודשים טסתי לרומא למשך שבוע. טסתי במטו...,f
6,אני כבר שנתיים נשוי והשנה אני ואישתי סוף סוף י...,m
7,השנה התחלנו שיפוץ בדירה שלנו בתל אביב. הדירה ה...,f


(753, 2)

In [28]:
df_test.head(3)
df_test.shape

,test_example_id,story
0,0,כל קיץ אני והמשפחה נוסעים לארצות הברית לוס אנג...
1,1,"הגעתי לשירות המדינה אחרי שנתיים כפעיל בתנועת ""..."
2,2,אחת האהבות הגדולות שלי אלו הכלבים שלי ושל אישת...


(323, 2)

### Your implementation:
Write your code solution in the following code-cells

In [29]:
# labaling the data to suitable df
def label_data(df, label):
    label.fit(df["gender"])
    df["gender"] = label.fit_transform(df["gender"])
    return df

In [31]:
# get the tokens
def get_tokens(text):
    tokens = ht.tokenize(text)
    list_of_words = [token for grp, token, token_num, (start_index, end_index) in tokens if grp == "HEBREW"]    
    return list_of_words

In [33]:
# calculating f1 score to predict the avarage f1
def calculate_f1(y_predict, y_true):
    f1_female = f1_score(y_pred = y_predict , y_true = y_test, pos_label=0)
    f1_male = f1_score(y_pred = y_predict, y_true = y_test, pos_label=1)
    avg_f1=(f1_male + f1_female) /2
    return avg_f1

In [35]:
#clean the data,then split it to train and test prameters the prepare the data
df_without_dup = df_train.drop_duplicates(subset="story")

label = LabelEncoder()
df_clean = label_data(df_without_dup, label)
y = df_clean["gender"]
X = df_clean.drop(labels="gender", axis=1)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.20)

vector = CountVectorizer(tokenizer=get_tokens, ngram_range=(1,2), max_features=10000, max_df=0.15)
pipe = Pipeline([('vector', vector), ('MNB', MultinomialNB())])
multi_pip = pipe.fit(X_train["story"], y_train)
y_pred = multi_pip.predict(X_train["story"])


In [36]:
#showing the data in a DataFrame
pd.DataFrame({'y_train':y_train, 'y_pred':y_pred}).head(10)
pd.DataFrame({'y_train':y_train, 'y_pred':y_pred}).tail(10)
pd.DataFrame({'y_train':y_train, 'y_pred':y_pred}).shape

,y_train,y_pred
597,1,1
131,1,1
44,1,1
70,0,0
674,0,0
208,1,1
345,0,0
625,1,1
135,1,1
483,1,1


,y_train,y_pred
469,1,1
121,1,1
617,1,1
20,1,1
703,0,0
71,1,1
106,1,1
270,1,1
437,0,0
102,1,1


(599, 2)

In [37]:
f1_score(y_train, y_pred, average = 'macro')

0.9977305190253735

### Save output to csv (optional)
After you're done save your output to the 'classification_results.csv' csv file.<br/>
We assume that the dataframe with your results contain the following columns:
* column 1 (left column): 'test_example_id'  - the same id associated to each of the test stories to be predicted.
* column 2 (right column): 'predicted_category' - the predicted gender value for each of the associated story. 

Assuming your predicted values are in the `df_predicted` dataframe, you should save you're results as following:

In [40]:
y_test_pred = multi_pip.predict(df_test["story"])
results = pd.DataFrame(
    {
        "id": [ _ for _ in range(len(y_test_pred))],
        "predict": label.inverse_transform(y_test_pred)
    }
)

results.to_csv("resultsofclassification.csv", index=False)